In [1]:
import random
import warnings
import numpy as np
import matplotlib.pyplot as plt
import dataLoader as dl
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import util as ut

#from Inner_Speech_Dataset.Plotting.ERPs import 
from Inner_Speech_Dataset.Python_Processing.Data_extractions import  Extract_data_from_subject
from Inner_Speech_Dataset.Python_Processing.Data_processing import  Select_time_window, Transform_for_classificator, Split_trial_in_time
from Inner_Speech_Dataset.Python_Processing.Data_processing import  Calculate_power_windowed
from Inner_Speech_Dataset.Python_Processing.Utilitys import picks_from_channels
from Inner_Speech_Dataset.Python_Processing.Data_processing import Average_in_frec

In [2]:


sampling_rate = 128
nr_of_datasets=8
data, labels = dl.load_multiple_datasets(nr_of_datasets=nr_of_datasets, sampling_rate=sampling_rate, t_min=2, t_max=3)

ch_names = ut.get_channelNames()

print(data.shape)
data_p =  ut.get_power_array(data[:,:128,:], sampling_rate, trialSplit=1).squeeze()
print(data_p.shape)
#print(data_p[:,:,1])

#Getting Freq Data 
nr_of_buckets = 5
buckets = ut.createFreqBuckets(data[:,:128,:], nr_of_buckets)
print("buckets")
print(buckets)
data_f = ut.data_into_freq_buckets(data[:,:128,:], nr_of_buckets, buckets)
#data_f = np.concatenate([dataf, np.zeros([data_f.shape[0], nr_of_datasets, data_f.shape[2]])], axis=1)

print(data_f.shape)


## Normalize data



Data shape: [trials x channels x samples]
(500, 128, 128)
Labels shape
(500, 4)
Final data shape
(100, 128, 128)
Final labels shape
(100,)
Up is 0.0 and Down is 1.0
Up is 0.0 and Down is 1.0
Up is 0.0 and Down is 1.0
Up is 0.0 and Down is 1.0
Up is 0.0 and Down is 1.0
Up is 0.0 and Down is 1.0
Up is 0.0 and Down is 1.0
(788, 128, 128)
(788, 128, 2)
buckets
[[ 0  3]
 [ 3  9]
 [ 9 21]
 [21 41]
 [40 63]]
(788, 128, 5)


In [38]:
print(data_f.shape)
print(data_p.shape)
print(labels.shape)
print(data_p[2,5])
data = np.concatenate([data_f, data_p, data], axis =2 )
print(data.shape)
data = keras.utils.normalize(data, axis=1, order=2)
print(data.shape)
#print(data[2,5])
#print(labels[2])
print(labels[:].shape)

(788, 128, 5)
(788, 128, 2)
(788, 2)
[1.01648499e-10 5.52805943e-11]
(788, 128, 135)
(788, 128, 135)
(788, 2)


In [40]:
#Splitting into training and test data
#print(labels)


order = np.arange(labels.shape[0])
np.random.shuffle(order)

temp_data = np.zeros(data.shape)
temp_labels = np.zeros(labels.shape)

for x in range(labels.shape[0]):
    i = order[x]
    
    temp_data[x] = data[i]
    temp_labels[x] = labels[i]

data = temp_data
labels = temp_labels

data_train, data_test = np.split(data, indices_or_sections=[int(labels.shape[0]*0.75)],axis=0)
labels_train, labels_test = np.split(labels, indices_or_sections=[int(labels.shape[0]*0.75)],axis=0)
print(labels_train.shape)
print(data_train.shape)
print(data_test.shape)




(591, 2)
(591, 128, 135)
(197, 128, 135)


In [53]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

tf.keras.backend.clear_session()





eeg_model = tf.keras.Sequential([
    

    layers.LocallyConnected1D(56, input_shape = (data_train.shape[1],data_train.shape[2]), 
     kernel_size=30, 
     padding="valid", strides=2,
     activation="relu"),
    
    layers.Dropout(0.7),
    
    layers.LocallyConnected1D(10, 
    kernel_size=20, 
    padding="valid", strides=1,
    activation="relu"),
    
    layers.Dropout(0.8),
    
    layers.Dense(units=200, activation="relu"),
    
    layers.Dropout(0.8),

    layers.Flatten(),

    layers.Dense(units=2, activation="softmax")


])
eeg_model.build()
eeg_model.summary()



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 locally_connected1d (Locall  (None, 50, 56)           11342800  
 yConnected1D)                                                   
                                                                 
 dropout (Dropout)           (None, 50, 56)            0         
                                                                 
 dense (Dense)               (None, 50, 500)           28500     
                                                                 
 dropout_1 (Dropout)         (None, 50, 500)           0         
                                                                 
 locally_connected1d_1 (Loca  (None, 31, 10)           3100310   
 llyConnected1D)                                                 
                                                                 
 dropout_2 (Dropout)         (None, 31, 10)            0

In [54]:
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
#callback2 = tf.keras.callbacks.EarlyStopping(monitor='accuracy', patience=10, restore_best_weights=True)
#tensorboard_callback = tf.keras.callbacks.TensorBoard(histogram_freq=2)

eeg_model.compile(optimizer='adam',
              loss="binary_crossentropy",
              metrics=['accuracy'])

data_train_send = np.reshape(data_train, [data_train.shape[0], 1 , data_train.shape[1], data_train.shape[2], 1])
#data_train_send = np.reshape(data_train, [data_train.shape[0], 1 , data_train.shape[1], data_train.shape[2], 1])

data_test_send = np.reshape(data_test, [data_test.shape[0], 1 , data_test.shape[1], data_test.shape[2], 1])
#data_test_send = np.reshape(data_test, [data_test.shape[0], 1 , data_test.shape[1], data_test.shape[2], 1])

outputs = eeg_model.fit(data_train, labels_train, validation_split=0.1 ,callbacks=[callback], epochs=50) #

print("Results")
eeg_model.evaluate(data_test, labels_test)
result = eeg_model.predict(data_test)

Epoch 1/50
17/17 [==============================] - 5s 214ms/step - loss: 0.7037 - accuracy: 0.4953 - val_loss: 0.6959 - val_accuracy: 0.4500
Epoch 2/50
17/17 [==============================] - 3s 195ms/step - loss: 0.7103 - accuracy: 0.5141 - val_loss: 0.6942 - val_accuracy: 0.4333
Epoch 3/50
17/17 [==============================] - 3s 193ms/step - loss: 0.7142 - accuracy: 0.4972 - val_loss: 0.6975 - val_accuracy: 0.4667
Epoch 4/50
 5/17 [=======>......................] - ETA: 2s - loss: 0.7181 - accuracy: 0.5188

KeyboardInterrupt: 

In [46]:

eeg_model.save('saved_model/my_model6')

INFO:tensorflow:Assets written to: saved_model/my_model6\assets


In [47]:
print("Results")
eeg_model.evaluate(data_test, labels_test)
result = eeg_model.predict(data_test)

result2 = []
print(result)
for res in result:
    x = np.maximum(res[0], res[1])
    result2.append(int(np.where(res == x)[0]))

result2 = np.array(result2)
print(result2)

Results
7/7 [==============================] - 1s 123ms/step
[[0.47737655 0.52262354]
 [0.38966712 0.6103329 ]
 [0.47881296 0.521187  ]
 [0.47822347 0.52177656]
 [0.4693499  0.53065014]
 [0.4776438  0.5223562 ]
 [0.47772625 0.5222737 ]
 [0.4635916  0.5364084 ]
 [0.47336978 0.5266303 ]
 [0.46468732 0.53531265]
 [0.47645646 0.5235436 ]
 [0.476968   0.523032  ]
 [0.47442225 0.5255778 ]
 [0.47561613 0.52438384]
 [0.462317   0.537683  ]
 [0.4646464  0.53535354]
 [0.3883958  0.6116042 ]
 [0.47151524 0.5284848 ]
 [0.45204785 0.5479521 ]
 [0.47794533 0.5220547 ]
 [0.43207544 0.56792456]
 [0.45140547 0.5485946 ]
 [0.47822794 0.521772  ]
 [0.4631676  0.53683245]
 [0.47533414 0.5246659 ]
 [0.4682925  0.5317075 ]
 [0.47703227 0.5229677 ]
 [0.47797903 0.52202094]
 [0.42021608 0.5797839 ]
 [0.46835703 0.53164303]
 [0.47778246 0.52221763]
 [0.47732425 0.52267575]
 [0.45689324 0.54310673]
 [0.46090165 0.5390983 ]
 [0.41844654 0.5815534 ]
 [0.46778393 0.532216  ]
 [0.42523128 0.5747688 ]
 [0.4592637  0